In [94]:
from tensorflow import keras
from PIL import Image
from torchvision import models, transforms
import torch.nn as nn
import torch.optim as optim
import torch
import random
from torchvision import models, transforms
from numpy.linalg import norm
import numpy as np
import math
import time
from torch.utils.data import DataLoader, TensorDataset

In [95]:
import sys
root = '../../'
sys.path.append(root)
from HelpfulFunctions.batchCreation import createBatch
from HelpfulFunctions.metrics import meanAveragePrecision
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [96]:
X_train = np.load( root + "Features/train_features_vgg16_cifar10.npy" ) # Shape = (45000, 4096)
X_train_tensor = torch.tensor(X_train)
y_train = np.load( root + "Features/train_labels_vgg16_cifar10.npy" ) # Shape = (45000,)


X_test = np.load( root + "Features/test_features_vgg16_cifar10.npy" ) # Shape = (10000, 4096)
X_test_tensor = torch.tensor(X_test)
y_test = np.load( root + "Features/test_labels_vgg16_cifar10.npy" ) # Shape = (10000,)


In [97]:
def CreateDataset(root, num_classes, batch_size, train = True):
    if train == True:
        #Create X_train_tensor
        X_train = np.load( root + "Features/train_features_vgg16_cifar10.npy" ) # Shape = (45000, 4096)
        X_train_tensor = torch.tensor(X_train)

        #Create Y_train_tensor
        y_train = np.load( root + "Features/train_labels_vgg16_cifar10.npy" ) # Shape = (45000,)
        y_train_tensor = torch.tensor(y_train, dtype=torch.long)
        y_train_tensor = torch.nn.functional.one_hot(y_train_tensor, num_classes) #One-Hot Encoded -> Shape = (45000, num_classes)

        #Create indices
        indices = torch.arange(len(X_train_tensor))

        dataset = TensorDataset(X_train_tensor, y_train_tensor, indices)
        train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        return train_loader

    #Missing implementation for Test and Validation

In [98]:
train_loader = CreateDataset(root, num_classes = 10, batch_size = 128)

In [99]:
class CustomNN(nn.Module):
    def __init__(self):
        super(CustomNN, self).__init__()
        self.fc_layers = nn.Sequential(
            nn.Linear(4096, 1024),  # First fully connected layer
            nn.ReLU(),
            nn.Linear(1024, 48),    # Second fully connected layer to reduce to 4000
        )

        # Initialize weights and biases from gaussian distribution
        for layer in self.fc_layers:
            if isinstance(layer, nn.Linear):
                nn.init.normal_(layer.weight, mean=0.0, std=0.01)  # Initialize weights based on paper
                nn.init.normal_(layer.bias, mean=0.0, std=0.01)    # Initialize biases based on paper

    def forward(self, x):
        return self.fc_layers(x)

In [100]:
model = CustomNN().to(device)

In [101]:
class DPSHLoss(torch.nn.Module):
    def __init__(self, train_size, n_classes, bit):
        super(DPSHLoss, self).__init__()
        self.U = torch.zeros(train_size, bit).float().to(device)
        self.Y = torch.zeros(train_size, n_classes).float().to(device)

    def forward(self, u, y, ind, eta):
        self.U[ind, :] = u.data
        self.Y[ind, :] = y.float()

        s = (y @ self.Y.t() > 0).float()
        inner_product = u @ self.U.t() * 0.5

        likelihood_loss = (1 + (-(inner_product.abs())).exp()).log() + inner_product.clamp(min=0) - s * inner_product

        likelihood_loss = likelihood_loss.mean()

        quantization_loss = eta * (u - u.sign()).pow(2).mean()

        return likelihood_loss + quantization_loss

In [ ]:
def train_val(device, X_train_tensor, y_train, train_size, batch_size, n_classes, bit, num_epoch, lr):

    num_batches = train_size / batch_size

    optimizer = optim.Adam(model.parameters(), lr=0.001)

    criterion = DPSHLoss(train_size, n_classes, bit)


    #model.train()

    #Best_mAP = 0

    for epoch in range(num_epoch):

        current_time = time.strftime('%H:%M:%S', time.localtime(time.time()))

        print("%s[%2d/%2d][%s] bit:%d, dataset:%s, training...." % (
            "DPSH", epoch + 1, num_epoch, current_time, bit, "CIFAR"), end="")

        train_loss = 0
        for image, label, ind in train_loader:
            image = image.to(device)
            label = label.to(device)

            optimizer.zero_grad()
            u = model(image)


            loss = criterion(u, label.float(), ind, lr)
            train_loss += loss.item()

            loss.backward()
            optimizer.step()

        train_loss = train_loss / (train_size / batch_size)

        print("\b\b\b\b\b\b\b loss:%.5f" % (train_loss))

        #if (epoch + 1) % config["test_map"] == 0:
            #Best_mAP = validate(config, Best_mAP, test_loader, dataset_loader, net, bit, epoch, num_dataset)


In [103]:
X_batch, y_batch, index = createBatch(X_train_tensor, y_train, batch_size = 128, oneHot = 10)
print("X_batch shape:", X_batch.shape)  # Should be (batch_size, num_features)
print("y_batch shape:", y_batch.shape)  # Should be (batch_size, num_classes)
print("index:", index)    

X_batch shape: torch.Size([128, 4096])
y_batch shape: torch.Size([128, 10])
index: [24537, 3169, 28218, 19101, 33748, 22950, 24920, 43712, 12844, 14679, 29841, 25541, 18170, 39498, 22498, 43331, 22797, 31116, 43760, 28106, 35695, 5553, 44256, 18495, 41903, 21266, 43520, 22004, 11852, 29611, 37479, 1833, 14885, 34272, 31900, 39101, 8938, 19987, 16481, 3501, 20774, 25222, 44651, 11930, 18174, 21043, 19311, 38530, 16773, 9039, 19364, 41085, 30653, 11661, 37551, 24866, 15791, 16285, 30489, 27225, 13307, 14659, 27826, 19397, 11492, 44101, 38720, 21494, 36214, 41854, 37268, 22770, 22218, 17015, 32168, 28242, 37728, 16717, 26209, 15731, 9232, 20525, 2302, 1191, 8831, 8040, 24937, 35759, 30582, 41430, 17203, 40685, 42114, 23253, 9597, 29584, 13539, 42145, 24903, 4428, 38802, 980, 15339, 42010, 15444, 38427, 32480, 27259, 15351, 17766, 21811, 18229, 38711, 24572, 19504, 27285, 11878, 30917, 14630, 44192, 29624, 42711, 42614, 2192, 34469, 42475, 20635, 6435]


In [104]:
train_val(device, X_train_tensor, y_train, train_size = 45000, batch_size = 128, n_classes = 10, bit = 48, num_epoch = 150, lr = 0.1)

DPSH[ 1/150][21:15:07] bit:48, dataset:CIFAR, training... loss:0.79733
DPSH[ 2/150][21:15:09] bit:48, dataset:CIFAR, training... loss:0.77960
DPSH[ 3/150][21:15:12] bit:48, dataset:CIFAR, training... loss:0.78334
DPSH[ 4/150][21:15:14] bit:48, dataset:CIFAR, training... loss:0.78391
DPSH[ 5/150][21:15:17] bit:48, dataset:CIFAR, training... loss:0.78409
DPSH[ 6/150][21:15:19] bit:48, dataset:CIFAR, training... loss:0.78402
DPSH[ 7/150][21:15:21] bit:48, dataset:CIFAR, training... loss:0.78449
DPSH[ 8/150][21:15:24] bit:48, dataset:CIFAR, training... loss:0.78555
DPSH[ 9/150][21:15:26] bit:48, dataset:CIFAR, training... loss:0.78661
DPSH[10/150][21:15:29] bit:48, dataset:CIFAR, training... loss:0.78781
DPSH[11/150][21:15:32] bit:48, dataset:CIFAR, training... loss:0.78895
DPSH[12/150][21:15:35] bit:48, dataset:CIFAR, training... loss:0.78911
DPSH[13/150][21:15:38] bit:48, dataset:CIFAR, training... loss:0.78881
DPSH[14/150][21:15:41] bit:48, dataset:CIFAR, training... loss:0.78942
DPSH[1

KeyboardInterrupt: 